<a href="https://colab.research.google.com/github/wenjing89/json/blob/main/json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

for index, d in enumerate(data['features']):
    if index >= 10:
        break
    print(d)

{'type': 'Feature', 'properties': {'id': '5901010076', 'name': '榮星藥局', 'phone': '(02)27124696', 'address': '臺北市松山區南京東路４段１３３巷５弄１號', 'mask_adult': 0, 'mask_child': 2490, 'updated': '2022/03/09 22:53:23', 'available': '星期一上午看診、星期二上午看診、星期三上午看診、星期四上午看診、星期五上午看診、星期六上午看診、星期日上午看診、星期一下午看診、星期二下午看診、星期三下午看診、星期四下午看診、星期五下午看診、星期六下午看診、星期日下午看診、星期一晚上看診、星期二晚上看診、星期三晚上看診、星期四晚上看診、星期五晚上看診、星期六晚上看診、星期日晚上休診', 'note': '-', 'custom_note': '', 'website': '', 'county': '臺北市', 'town': '松山區', 'cunli': '東勢里', 'service_periods': 'NNNNNNNNNNNNNNNNNNNNY'}, 'geometry': {'type': 'Point', 'coordinates': [121.555235, 25.052137]}}
{'type': 'Feature', 'properties': {'id': '5901010085', 'name': '安世藥局', 'phone': '(02)27133405', 'address': '臺北市松山區南京東路４段１３３巷５弄９號', 'mask_adult': 750, 'mask_child': 1650, 'updated': '2022/03/09 22:53:23', 'available': '星期一上午看診、星期二上午看診、星期三上午看診、星期四上午看診、星期五上午看診、星期六上午看診、星期日上午看診、星期一下午看診、星期二下午看診、星期三下午看診、星期四下午看診、星期五下午看診、星期六下午看診、星期日下午看診、星期一晚上看診、星期二晚上看診、星期三晚上看診、星期四晚上看診、星期五晚上看診、星期六晚上看診、星期日晚上休診', 'note': '09:00~

In [3]:
# 計算各地區的藥局數量
med_count = {}

# 填入欄位名稱 #data['features'] 是一個特徵的列表，每個特徵都包含一個屬性字典，其中 'county' 是一個特定的屬性，代表了特徵所在的區域或縣市。因此，d['properties']['county'] 可以用來獲取每個特徵的區域或縣市名稱
for d in data['features']:
    conunty = d['properties']['county']
    if conunty not in med_count:
         med_count[conunty] = 0
    else:
        med_count[conunty] += 1

print(med_count)
# {'台北市': 123, '新北市': 456 ...}

{'臺北市': 338, '': 48, '高雄市': 421, '臺中市': 419, '臺南市': 270, '基隆市': 56, '新竹市': 38, '嘉義市': 66, '新北市': 500, '桃園市': 263, '新竹縣': 45, '宜蘭縣': 75, '苗栗縣': 55, '彰化縣': 178, '南投縣': 66, '雲林縣': 128, '嘉義縣': 83, '屏東縣': 139, '澎湖縣': 10, '花蓮縣': 45, '臺東縣': 22, '金門縣': 5, '連江縣': 0}


In [4]:
# 每個地區的成人剩餘口罩數量，並且將結果從大到小排列
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    conunty = d['properties']['county']
    # mask_adult = d['properties']['mask_adult']
    if conunty not in mask_count:
        mask_count[conunty] = 0
    else:
        mask_count[conunty] = mask_count[conunty] + d['properties']['mask_adult']

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse = True))

print(mask_count)
# {'台北市': 12345, '新北市': 45678 ...}

{'新北市': 670150, '臺中市': 535430, '高雄市': 424170, '臺南市': 392430, '桃園市': 330010, '臺北市': 322490, '彰化縣': 218090, '雲林縣': 195540, '屏東縣': 181480, '苗栗縣': 119500, '嘉義縣': 112190, '嘉義市': 105870, '宜蘭縣': 105830, '南投縣': 81680, '基隆市': 79840, '': 68040, '花蓮縣': 56570, '新竹縣': 51280, '新竹市': 45630, '臺東縣': 27720, '金門縣': 15510, '澎湖縣': 11350, '連江縣': 0}


In [10]:
# 將昨天計算的「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」
import sqlite3
import datetime

conn = sqlite3.connect('mask.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料
for city, counts in med_count.items():
  t = datetime.datetime.now()
  c.execute("INSERT INTO pharmacies VALUES (?,?,?)", (city, counts, t))
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

[('臺北市', '338', '2024-04-24 07:43:26.673321'), ('', '48', '2024-04-24 07:43:26.685269'), ('高雄市', '421', '2024-04-24 07:43:26.694963'), ('臺中市', '419', '2024-04-24 07:43:26.704608'), ('臺南市', '270', '2024-04-24 07:43:26.714342'), ('基隆市', '56', '2024-04-24 07:43:26.723779'), ('新竹市', '38', '2024-04-24 07:43:26.733333'), ('嘉義市', '66', '2024-04-24 07:43:26.742994'), ('新北市', '500', '2024-04-24 07:43:26.752558'), ('桃園市', '263', '2024-04-24 07:43:26.762244'), ('新竹縣', '45', '2024-04-24 07:43:26.772166'), ('宜蘭縣', '75', '2024-04-24 07:43:26.781157'), ('苗栗縣', '55', '2024-04-24 07:43:26.790556'), ('彰化縣', '178', '2024-04-24 07:43:26.800234'), ('南投縣', '66', '2024-04-24 07:43:26.810401'), ('雲林縣', '128', '2024-04-24 07:43:26.819961'), ('嘉義縣', '83', '2024-04-24 07:43:26.829499'), ('屏東縣', '139', '2024-04-24 07:43:26.838492'), ('澎湖縣', '10', '2024-04-24 07:43:26.847737'), ('花蓮縣', '45', '2024-04-24 07:43:26.857338'), ('臺東縣', '22', '2024-04-24 07:43:26.867816'), ('金門縣', '5', '2024-04-24 07:43:26.877655'), ('連江

In [11]:
# 接下來請將昨天計算的「各地區的剩餘口罩」存入資料庫的 `masks` 表格，欄位包含「地區」、「剩餘口罩數量」、「新增時間」
import sqlite3
import datetime

conn = sqlite3.connect('mask.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (address text, remaining text, addtime datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 新增資料
for address, remaining in mask_count.items():
  t = datetime.datetime.now()
  c.execute("INSERT INTO masks VALUES (?,?,?)", (address, remaining, t))
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())

conn.commit()
conn.close()

[('新北市', '2680960', '2024-04-24 07:49:56.729992'), ('臺中市', '2145560', '2024-04-24 07:49:56.738665'), ('高雄市', '1696680', '2024-04-24 07:49:56.747403'), ('臺南市', '1574130', '2024-04-24 07:49:56.756573'), ('桃園市', '1325830', '2024-04-24 07:49:56.766108'), ('臺北市', '1289960', '2024-04-24 07:49:56.776477'), ('彰化縣', '876290', '2024-04-24 07:49:56.786570'), ('雲林縣', '786510', '2024-04-24 07:49:56.796336'), ('屏東縣', '734230', '2024-04-24 07:49:56.805888'), ('苗栗縣', '512740', '2024-04-24 07:49:56.816755'), ('嘉義縣', '451730', '2024-04-24 07:49:56.827264'), ('嘉義市', '429390', '2024-04-24 07:49:56.837244'), ('宜蘭縣', '426530', '2024-04-24 07:49:56.846816'), ('南投縣', '355400', '2024-04-24 07:49:56.857139'), ('基隆市', '320140', '2024-04-24 07:49:56.867133'), ('', '275670', '2024-04-24 07:49:56.876892'), ('花蓮縣', '228560', '2024-04-24 07:49:56.887502'), ('新竹縣', '205330', '2024-04-24 07:49:56.896584'), ('新竹市', '182520', '2024-04-24 07:49:56.905154'), ('臺東縣', '113250', '2024-04-24 07:49:56.915008'), ('金門縣', '62040',